<a href="https://colab.research.google.com/github/cesaroliveiragoes/Machine-Learning-Data-Science-Classes/blob/main/Machine_Learning_and_Data_Science__Exs_06_pt_3_Autoregressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# INTEGRANTE
# NOME: CESAR OLIVEIRA GOES
# RM: 98119

In [ ]:
# Carga de bibliotecas
!pip install pandas scikit-learn
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as MAE
import statsmodels.api as sm

# Carga de dados
df = pd.read_csv("https://raw.githubusercontent.com/diogenesjusto/FIAP/master/SHIFT/Data/pib.csv")
df

,Unnamed: 0,ANO_MES,PIB,BRL,BRP,BRT,SLP,SPP,SPT,PRL,...,PIBi3,PIBi4,PIBi5,PIBi6,PIBi7,PIBi8,PIBi9,PIBi10,PIBi11,PIBi12
0,1,jan/04,103.09,109.19,108.67,109.08,102.84,114.27,105.38,127.49,...,106.27,104.52,102.59,102.24,99.96,101.93,101.17,101.70,100.03,100.00
1,2,fev/04,102.05,95.65,104.52,97.63,90.76,109.83,94.99,96.60,...,104.10,106.27,104.52,102.59,102.24,99.96,101.93,101.17,101.70,100.03
2,3,mar/04,110.43,91.69,125.53,99.26,89.78,133.24,99.43,83.79,...,103.91,104.10,106.27,104.52,102.59,102.24,99.96,101.93,101.17,101.70
3,4,abr/04,106.77,95.36,118.34,100.49,94.58,123.55,101.02,91.79,...,103.09,103.91,104.10,106.27,104.52,102.59,102.24,99.96,101.93,101.17
4,5,mai/04,108.08,92.47,121.49,98.96,91.34,128.32,99.55,85.73,...,102.05,103.09,103.91,104.10,106.27,104.52,102.59,102.24,99.96,101.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,134,fev/15,144.42,147.35,133.73,143.57,147.81,148.96,147.54,157.30,...,151.46,156.65,154.04,154.75,154.34,148.92,154.49,150.92,151.44,148.92
134,135,mar/15,151.68,149.64,163.27,152.09,154.87,179.07,159.56,147.28,...,150.70,151.46,156.65,154.04,154.75,154.34,148.92,154.49,150.92,151.44
135,136,abr/15,147.03,153.81,149.93,152.28,161.21,164.74,161.16,162.21,...,149.51,150.70,151.46,156.65,154.04,154.75,154.34,148.92,154.49,150.92
136,137,mai/15,148.94,149.66,153.03,149.82,157.03,169.39,158.76,149.06,...,144.42,149.51,150.70,151.46,156.65,154.04,154.75,154.34,148.92,154.49


In [ ]:
# Construir colunas com lag para PIB
df['PIBi1_'] = df['PIB'].shift(1)
df['PIBi2_'] = df['PIB'].shift(2)
df['PIBi3_'] = df['PIB'].shift(3)
df['PIBi4_'] = df['PIB'].shift(4)
df['PIBi5_'] = df['PIB'].shift(5)
df['PIBi6_'] = df['PIB'].shift(6)


df.head(15)

,Unnamed: 0,ANO_MES,PIB,BRL,BRP,BRT,SLP,SPP,SPT,PRL,...,PIBi9,PIBi10,PIBi11,PIBi12,PIBi1_,PIBi2_,PIBi3_,PIBi4_,PIBi5_,PIBi6_
0,1,jan/04,103.09,109.19,108.67,109.08,102.84,114.27,105.38,127.49,...,101.17,101.70,100.03,100.00,NaN,NaN,NaN,NaN,NaN,NaN
1,2,fev/04,102.05,95.65,104.52,97.63,90.76,109.83,94.99,96.60,...,101.93,101.17,101.70,100.03,103.09,NaN,NaN,NaN,NaN,NaN
2,3,mar/04,110.43,91.69,125.53,99.26,89.78,133.24,99.43,83.79,...,99.96,101.93,101.17,101.70,102.05,103.09,NaN,NaN,NaN,NaN
3,4,abr/04,106.77,95.36,118.34,100.49,94.58,123.55,101.02,91.79,...,102.24,99.96,101.93,101.17,110.43,102.05,103.09,NaN,NaN,NaN
4,5,mai/04,108.08,92.47,121.49,98.96,91.34,128.32,99.55,85.73,...,102.59,102.24,99.96,101.93,106.77,110.43,102.05,103.09,NaN,NaN
5,6,jun/04,109.19,89.25,119.93,96.11,87.79,128.51,96.83,84.00,...,104.52,102.59,102.24,99.96,108.08,106.77,110.43,102.05,103.09,NaN
6,7,jul/04,111.95,100.39,125.64,106.03,100.22,135.26,108.01,98.32,...,106.27,104.52,102.59,102.24,109.19,108.08,106.77,110.43,102.05,103.09
7,8,ago/04,110.92,94.20,129.67,102.13,93.66,139.38,103.81,87.53,...,104.10,106.27,104.52,102.59,111.95,109.19,108.08,106.77,110.43,102.05
8,9,set/04,110.18,95.54,126.70,102.50,94.93,138.84,104.68,91.66,...,103.91,104.10,106.27,104.52,110.92,111.95,109.19,108.08,106.77,110.43
9,10,out/04,109.71,99.46,124.85,105.14,98.58,136.18,106.93,93.66,...,103.09,103.91,104.10,106.27,110.18,110.92,111.95,109.19,108.08,106.77


In [ ]:
# MODELO AUTOREGRESSIVO COM PIB
# Separação de treino e teste
treino = df.iloc[6:126]
teste = df.iloc[126:138]

# Regressão
Y_treino = treino.iloc[:,[2]]

#Modelo auto regressivo
X = ['PIBi1_', 'PIBi2_','PIBi3_','PIBi4_','PIBi5_','PIBi6_']
X_treino = treino.loc[:,X]
mod = LinearRegression()
mod.fit(X_treino, Y_treino)
# Análise de coeficientes
print(mod.coef_)
print(mod.intercept_)

# Cálculo de erro com python
X_teste = teste.loc[:,X]
Y_prev = mod.predict(X_teste)
Y_real = teste.loc[:,['PIB']]
Erro = Y_prev - Y_real

# Erro de previsão
print(MAE(Y_real, Y_prev))

[[ 0.62137887  0.5033289  -0.03872748 -0.35125478  0.19867779  0.04112256]]
[3.85723684]
2.2474902460562425


In [ ]:
# Construir colunas com lag para BRL
df['BRLi1_'] = df['BRL'].shift(1)
df['BRLi2_'] = df['BRL'].shift(2)
df['BRLi3_'] = df['BRL'].shift(3)
df['BRLi4_'] = df['BRL'].shift(4)
df['BRLi5_'] = df['BRL'].shift(5)
df['BRLi6_'] = df['BRL'].shift(6)


df.head(15)

,Unnamed: 0,ANO_MES,PIB,BRL,BRP,BRT,SLP,SPP,SPT,PRL,...,PIBi3_,PIBi4_,PIBi5_,PIBi6_,BRLi1_,BRLi2_,BRLi3_,BRLi4_,BRLi5_,BRLi6_
0,1,jan/04,103.09,109.19,108.67,109.08,102.84,114.27,105.38,127.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,fev/04,102.05,95.65,104.52,97.63,90.76,109.83,94.99,96.60,...,NaN,NaN,NaN,NaN,109.19,NaN,NaN,NaN,NaN,NaN
2,3,mar/04,110.43,91.69,125.53,99.26,89.78,133.24,99.43,83.79,...,NaN,NaN,NaN,NaN,95.65,109.19,NaN,NaN,NaN,NaN
3,4,abr/04,106.77,95.36,118.34,100.49,94.58,123.55,101.02,91.79,...,103.09,NaN,NaN,NaN,91.69,95.65,109.19,NaN,NaN,NaN
4,5,mai/04,108.08,92.47,121.49,98.96,91.34,128.32,99.55,85.73,...,102.05,103.09,NaN,NaN,95.36,91.69,95.65,109.19,NaN,NaN
5,6,jun/04,109.19,89.25,119.93,96.11,87.79,128.51,96.83,84.00,...,110.43,102.05,103.09,NaN,92.47,95.36,91.69,95.65,109.19,NaN
6,7,jul/04,111.95,100.39,125.64,106.03,100.22,135.26,108.01,98.32,...,106.77,110.43,102.05,103.09,89.25,92.47,95.36,91.69,95.65,109.19
7,8,ago/04,110.92,94.20,129.67,102.13,93.66,139.38,103.81,87.53,...,108.08,106.77,110.43,102.05,100.39,89.25,92.47,95.36,91.69,95.65
8,9,set/04,110.18,95.54,126.70,102.50,94.93,138.84,104.68,91.66,...,109.19,108.08,106.77,110.43,94.20,100.39,89.25,92.47,95.36,91.69
9,10,out/04,109.71,99.46,124.85,105.14,98.58,136.18,106.93,93.66,...,111.95,109.19,108.08,106.77,95.54,94.20,100.39,89.25,92.47,95.36


In [ ]:
# MODELO AUTOREGRESSIVO COM BRL

# Separação de treino e teste
treino = df.iloc[6:126]
teste = df.iloc[126:138]

# Regressão
Y_treino = treino.iloc[:,[2]]

#Modelo auto regressivo
X = ['BRLi1_','BRLi2_','BRLi3_','BRLi4_','BRLi5_','BRLi6_']
X_treino = treino.loc[:,X]
mod = LinearRegression()
mod.fit(X_treino, Y_treino)
# Análise de coeficientes
print(mod.coef_)
print(mod.intercept_)

# Cálculo de erro com python
X_teste = teste.loc[:,X]
Y_prev = mod.predict(X_teste)
Y_real = teste.loc[:,['BRL']]
Erro = Y_prev - Y_real

# Erro de previsão
print(MAE(Y_real, Y_prev))

[[0.10395709 0.03199701 0.07956553 0.07186501 0.19419485 0.23847687]]
[45.06184343]
11.37763762705484


In [ ]:
# Construir colunas com lag para BRP
df['BRPi1_'] = df['BRP'].shift(1)
df['BRPi2_'] = df['BRL'].shift(2)
df['BRPi3_'] = df['BRL'].shift(3)
df['BRPi4_'] = df['BRL'].shift(4)
df['BRPi5_'] = df['BRL'].shift(5)
df['BRPi6_'] = df['BRL'].shift(6)


df.head(15)

,Unnamed: 0,ANO_MES,PIB,BRL,BRP,BRT,SLP,SPP,SPT,PRL,...,BRLi3_,BRLi4_,BRLi5_,BRLi6_,BRPi1_,BRPi2_,BRPi3_,BRPi4_,BRPi5_,BRPi6_
0,1,jan/04,103.09,109.19,108.67,109.08,102.84,114.27,105.38,127.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,fev/04,102.05,95.65,104.52,97.63,90.76,109.83,94.99,96.60,...,NaN,NaN,NaN,NaN,108.67,NaN,NaN,NaN,NaN,NaN
2,3,mar/04,110.43,91.69,125.53,99.26,89.78,133.24,99.43,83.79,...,NaN,NaN,NaN,NaN,104.52,109.19,NaN,NaN,NaN,NaN
3,4,abr/04,106.77,95.36,118.34,100.49,94.58,123.55,101.02,91.79,...,109.19,NaN,NaN,NaN,125.53,95.65,109.19,NaN,NaN,NaN
4,5,mai/04,108.08,92.47,121.49,98.96,91.34,128.32,99.55,85.73,...,95.65,109.19,NaN,NaN,118.34,91.69,95.65,109.19,NaN,NaN
5,6,jun/04,109.19,89.25,119.93,96.11,87.79,128.51,96.83,84.00,...,91.69,95.65,109.19,NaN,121.49,95.36,91.69,95.65,109.19,NaN
6,7,jul/04,111.95,100.39,125.64,106.03,100.22,135.26,108.01,98.32,...,95.36,91.69,95.65,109.19,119.93,92.47,95.36,91.69,95.65,109.19
7,8,ago/04,110.92,94.20,129.67,102.13,93.66,139.38,103.81,87.53,...,92.47,95.36,91.69,95.65,125.64,89.25,92.47,95.36,91.69,95.65
8,9,set/04,110.18,95.54,126.70,102.50,94.93,138.84,104.68,91.66,...,89.25,92.47,95.36,91.69,129.67,100.39,89.25,92.47,95.36,91.69
9,10,out/04,109.71,99.46,124.85,105.14,98.58,136.18,106.93,93.66,...,100.39,89.25,92.47,95.36,126.70,94.20,100.39,89.25,92.47,95.36


In [ ]:
# MODELO AUTOREGRESSIVO COM BRP

# Separação de treino e teste
treino = df.iloc[6:126]
teste = df.iloc[126:138]

# Regressão
Y_treino = treino.iloc[:,[2]]

#Modelo auto regressivo
X = ['BRPi1_','BRPi2_','BRPi3_','BRPi4_','BRPi5_','BRPi6_']
X_treino = treino.loc[:,X]
mod = LinearRegression()
mod.fit(X_treino, Y_treino)
# Análise de coeficientes
print(mod.coef_)
print(mod.intercept_)

# Cálculo de erro com python
X_teste = teste.loc[:,X]
Y_prev = mod.predict(X_teste)
Y_real = teste.loc[:,['BRP']]
Erro = Y_prev - Y_real

# Erro de previsão
print(MAE(Y_real, Y_prev))

[[ 0.38124082  0.10368682  0.08455961 -0.0252934   0.16388968  0.08755955]]
[28.33950249]
9.602371473681723


In [ ]:
# MODELO AUTOREGRESSIVO MULTIVARIADO COM BRL, BRP E PIB

# Separação de treino e teste
treino = df.iloc[1:126]
teste = df.iloc[126:138]

#Modelo auto regressivo
X = ['BRPi1_','BRLi1_','PIBi1_']
# Regressão
Y_treino = treino[['PIB']]


X_treino = treino.loc[:,X]
mod = LinearRegression()
mod.fit(X_treino, Y_treino)
# Análise de coeficientes
print(mod.coef_)
print(mod.intercept_)

# Cálculo de erro com python
X_teste = teste.loc[:,X]
Y_prev = mod.predict(X_teste)
Y_real = teste.loc[:,['PIB']]
Erro = Y_prev - Y_real

# Erro de previsão
print(MAE(Y_real, Y_prev))

[[-0.24621634 -0.09506053  1.35023481]]
[0.54240325]
2.1824720121787613
